In [ ]:
from sqlalchemy import create_engine
from pandas.io.sql import read_sql

conn_string = 'postgresql://iangow.me/crsp'
engine = create_engine(conn_string)

In [ ]:
tone_data = read_sql("SELECT category, word_list FROM lm_tone", engine)
tone_data = tone_data.set_index(['category'])

In [ ]:
import re

def make_regex(word_list):
    mod_word_list = [word.lower() for word in word_list]
    regex = '\\b(?:' + '|'.join(mod_word_list) + ')\\b'
    return  re.compile(regex)

tone_data

In [ ]:
def get_regex_matches(category, text):
    regex = make_regex(tone_data.ix[category]['word_list'])
    return re.findall(regex, text)

In [ ]:
file_name = '999419_T'

sql = """
    SELECT speaker_name, role, context,
        file_name, last_update, speaker_number, speaker_text
    FROM streetevents.speaker_data
    WHERE file_name='%s'
    ORDER BY context, speaker_number
""" % (file_name)

df = read_sql(sql, engine)
some_text = df["speaker_text"][3]
some_text

In [ ]:
def get_tone_data(text):
    text = text.lower()
    the_dict = {category: len(get_regex_matches(category, text))
                            for category in categories}
    return the_dict

In [ ]:
get_tone_data(some_text)

In [ ]:
df["tone_count"] = df["speaker_text"].map(get_tone_data)
df.drop(["speaker_name", "role", "speaker_text"], axis=1, inplace=True)

In [ ]:
# df

In [ ]:
tone_keys = df["tone_count"][0].keys()
tone_keys

In [ ]:
for key in tone_keys:
    df[key] = df["tone_count"].map(lambda x: x[key])
df.drop(["tone_count"], axis=1, inplace=True)

In [ ]:
df

In [ ]:
# Next write data to PostgreSQL